In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![](https://www.blumeglobal.com/wp-content/uploads/2018/11/NLP-image.jpg)

#  <h1>INTRODUCCIÓN A LOS DATOS SECUENCIALES Y AL ANÁLISIS DE TEXTO</h1>

El texto es un tipo de dato bastante particular, ya que fue diseñado para nosotros como humanos podamos conumicarnos. Sin embargo, las computadoras no entienden éste lenguaje (Primer problema). El analisis se vuelve tan complejo como el idioma en el que se esté trabaajando (Segundo problema). La buena noticia es que con la ayuda de técnica de análisis de datos y las mejoras en los algoritmos de procesamiento de lenguaje natural éstos problemas han quedado en el pasado. Dichas técnicas las exploraremos en el siguiente notebook hasta llegar a como detectar si un tweet es generado por una máquina o por un humano.

Antes de empezar descarguemos e importemos algunas dependecias.

In [ ]:
!python -m spacy download es_core_news_lg

In [ ]:
import pandas as pd
import re
import string
import operator
import matplotlib.pyplot as plt
import seaborn as sns
#import es_core_news_lg

from collections import defaultdict

In [ ]:
train_data = pd.read_csv('../input/aichallenge2020/training.csv')
test_data  = pd.read_csv('../input/aichallenge2020/test.csv')

Hay que conocer los datos..

In [ ]:
train_human_tweets = sum(train_data['is_organico'] == 1)
train_bot_tweets   = sum(train_data['is_organico'] == 0)

In [ ]:
plt.rcParams['figure.figsize'] = (7, 5)
fig = plt.figure()
sns.barplot(x=['Human Tweets', 'Bot Tweets'], y = [train_human_tweets, train_bot_tweets])
plt.title("Distribucón de los tweets en el set de entrnamiento")

Ésta primera vizualización no enseña que los datos están imbalanceados. Luego nos encargaremos de éste problema.

Miremos más a fondo.

In [ ]:
import numpy as np

def length(x):
    return len(x)

train_data['length'] = train_data['tweet'].apply(lambda x: length(x))

bot_tweet_length = train_data[train_data['is_organico'] == 0]['length']
human_tweet_length = train_data[train_data['is_organico'] == 1]['length']

print('Media de caracteres por tweet inorgánico: ', np.mean(list(bot_tweet_length)))
print('Media de caracteres por tweet orgánico: ', np.mean(list(human_tweet_length)))

plt.figure(figsize=(10,10))
plt.title('Bot and Human tweet length')
sns.distplot(human_tweet_length)
sns.distplot(bot_tweet_length)

Los tweets inorganicos son en promedio más largos, investiguemos esto más a fondo. La maxima cantidad de caracteres en un tweet es de 280 carácteres. Tomemos este valor como umbral. 

In [ ]:
tweet_length = 280

temp = train_data[train_data['is_organico'] == 0]
bot_data = temp[temp['length'] > tweet_length]

temp_ = train_data[train_data['is_organico'] == 1]
human_data = temp_[temp_['length'] > tweet_length]

x = ['Organicos', 'No Organicos']

plt.rcParams['figure.figsize'] = (7, 5)
fig = plt.figure()
sns.barplot(x=x, y = [human_data.shape[0], bot_data.shape[0]])
plt.title("tweets mas largos de 280 carácteres")
print(human_data.shape[0] + bot_data.shape[0])

La mayoría de los tweets más largos son inorganicos. Aumentemos el umbral.

In [ ]:
tweet_length = 300

temp = train_data[train_data['is_organico'] == 0]
bot_data = temp[temp['length'] > tweet_length]

temp_ = train_data[train_data['is_organico'] == 1]
human_data = temp_[temp_['length'] > tweet_length]

x = ['Organicos', 'No Organicos']

plt.rcParams['figure.figsize'] = (7, 5)
fig = plt.figure()
sns.barplot(x=x, y = [human_data.shape[0], bot_data.shape[0]])
plt.title("tweets mas largos de 300 carácteres")

In [ ]:
tweet_length = 10

temp = train_data[train_data['is_organico'] == 0]
bot_data = temp[temp['length'] < tweet_length]

temp_ = train_data[train_data['is_organico'] == 1]
human_data = temp_[temp_['length'] < tweet_length]

x = ['Organicos', 'No Organicos']

plt.rcParams['figure.figsize'] = (7, 5)
fig = plt.figure()
sns.barplot(x=x, y = [human_data.shape[0], bot_data.shape[0]])
plt.title("tweets mas largos de 300 carácteres")

Nuestro primer análisis podemos llegar a la conclusión que los tweets mas largos tienden a ser inorganicos, mientras que los tweets más cortos tienden a ser orgánicos.

Ahora chequeemos si todos los tweets están en español.

In [ ]:
!pip install langdetect

In [ ]:
from langdetect import detect

def detect_lang(text):
  try:
    return detect(text)
  except:
    return None

train_data['lang'] = train_data['tweet'].apply(lambda x: detect_lang(x))

In [ ]:
esp_tweet = train_data[train_data['lang'] == 'es']
no_esp_tweet = train_data[train_data['lang'] != 'es']

esp_tweet_qty = esp_tweet.shape[0]
no_esp_tweet_qty = no_esp_tweet.shape[0]

plt.rcParams['figure.figsize'] = (7, 5)
fig = plt.figure()
sns.barplot(x=['Tweets en Español', "Tweet en otro idioma"], y = [esp_tweet_qty,no_esp_tweet_qty])
plt.title("Languajes")

Tnemos algunos tweets que no están en español. Será importante tomar esto en cuenta.

De nuestro primer análisis exploratorio podemos concluir algunas cosas.

* Los datos se encuentran imbalanceados.
* Los tweets mas largos tienden a ser no organicos mientras que los más cortos tienden a ser orgánicos.

De ésta última conclusión podemos hacernos unas preguntas: 

**Los tweets tienden una cantidad máxima de carácteres de 280 carácteres, hay tweets con muchos más carácteres esto afecta la distribución del dataset. Ésto afectará el desempeño del modelo en entornos reales.**

**Hay tweets en Inglés y en Español si el modelo estará expuesto a datos en multi-idioma no hay suficientes tweets en otro idioma, ésto afectaría el performance en producción**

* La recolección de los datos no ha sido la mejor, éste es el stage más importante de la construcción de modelos de IA. La calidad de los datos no es buena.

* Los tweets necesitarán de Data Cleanning, para eliminar caracteres especiales y otros datos que sean inncesarios.

> #  <h1>LIMPIEZA DE DATOS</h1>

La parte más compleja de trabajar con texto es que lo que para nosotros puede ser muy útil, para un modelo es solo ruido, por tal motivo es necesario remover todo ese ruido en los datos, lo cual mejorará el performance final del modelo.

**¿Cuales son esos ruidos?** 

Caracteres especiales, palabras cortas y otras que solo entrenando al modelo se podrán identificar.

> #  <h1>NORMALIZACIÓN DE TEXTO</h1>

No solo los carácteres espeicales son ruido, sino las cojungaciones de verbos, palabras muy largas y conectivos, realizar este proceso tenemos... 

Lematizacion, Stemming y Stop Words

**Lematizacion**: La lematización es un proceso lingüístico que consiste en, dada una forma flexionada, hallar el lema correspondiente. El lema es la forma que por convenio se acepta como representante de todas las formas flexionadas de una misma palabra https://es.wikipedia.org/wiki/Lematizaci%C3%B3n

**Stemming**: Stemming es un método para reducir una palabra a su raíz o a un stem. Hay algunos algoritmos de stemming que ayudan en sistemas de recuperación de información. Stemming aumenta el recall que es una medida sobre el número de documentos que se pueden encontrar con una consulta. https://es.wikipedia.org/wiki/Stemming

**Stop words (Palabras Vacias)**: Palabras vacías es el nombre que reciben las palabras sin significado como artículos, pronombres, preposiciones, etc. que son filtradas antes o después del procesamiento de datos en lenguaje natural. https://es.wikipedia.org/wiki/Palabra_vac%C3%ADa

In [ ]:
print(train_data.head(20))
print(' ')
print('-----------------------------------------------------------------------------------------------')
print('-----------------------------------------------------------------------------------------------')
print('-----------------------------------------------------------------------------------------------')

def normalise_text(text):
    text = text.lower().strip() # lowercase
    text = re.sub(r'[0-9]', " numero ", text)
    text = re.sub(r"#\S+","tema", text) # replaces hashtags
    text = re.sub(r'http\S+', 'enlace', text)  # remove URL addresses
    text = re.sub(r"@\S+","nombre", text)
    text = re.sub(f"[{re.escape(string.punctuation)}]", ' ', text)
    text = re.sub(r"[^A-Za-z0-9ñ()!?\'\`\"]", " ", text)
    text = re.sub(r"\s{2,}", " ", text)
    return text
   

def remove_accents(text):
    text = re.sub(u"[á]", "a", text)
    text = re.sub(u"[é]", "e", text)
    text = re.sub(u"[í]", "i", text)
    text = re.sub(u"[ó]", "o", text)
    text = re.sub(u"[ú]", "u", text)
    return text
    

    
def remove_spaces(x):
    try:
        return ' '.join([word for word in x.split(' ') if word != ''])
    except:
        return None
    

def all_prep(text):
  text = remove_accents(text)
  text = normalise_text(text)
  text = remove_spaces(text)
  return text

train_data['tweet'] = train_data['tweet'].apply(lambda x: all_prep(x))
train_data

Ahora a manera de Baseline hagamos en una prueba, con una libreroa creada por Facebook llamada Fasttext, con ella podremos crear rapidamente, y además entrenar un modelo clasificador de texto.

In [ ]:
import fasttext as ft
from sklearn.metrics import accuracy_score

In [ ]:
rate = 0.3

ratio = int(len(train_data) * rate)

test = train_data[:ratio]
train = train_data[ratio:]

Para crear el clasificador, la funcion de entrenamiento toma un archivo de texto con el datapoint, cada linea del archivo de texto tendrá el datapoint con su respectiva etiqueta.

In [ ]:
train_dicty = {
    "data": []
}    

test_dicty = {
    "data": []
}    

for line in train_data.iterrows():
    label = line[1]['is_organico']
    text = line[1]['tweet']
    for_write = "__label__" + str(label) + " " + str(text) + "\n"
    train_dicty['data'].append(for_write)
    
for line in train_data.iterrows():
    label = line[1]['is_organico']
    text = line[1]['tweet']
    for_write = "__label__" + str(label) + " " + str(text) + "\n"
    test_dicty['data'].append(for_write)

In [ ]:
training_file = open('train_data.txt','w')
training_file.writelines(dicty['data'])
training_file.close()

test_file = open('test_data.txt','w')
test_file.writelines(dicty['data'])
test_file.close()

In [ ]:
classifier1 = ft.FastText.train_supervised('train_data.txt', lr=0.01, wordNgrams=1, epoch=15)
classifier2 = ft.FastText.train_supervised('train_data.txt', lr=0.001, wordNgrams=2, epoch=15)
classifier3 = ft.FastText.train_supervised('train_data.txt', lr=0.01, wordNgrams=3, epoch=100)

In [ ]:
print(classifier3.test("test_data.txt"))
print(classifier2.test("test_data.txt"))
print(classifier1.test("test_data.txt"))

In [ ]:
print('Metricas Clasificador #1: ', classifier1)
print('Metricas Clasificador #2: ', classifier2)
print('Metricas Clasificador #3: ', classifier3)

Asi de sencillo podemos crear un clasificador y hacer pruebas acerca de nuestro preprocesamiento de texto.

Todas las pruebas no estarán en éste notebook, por tal motivo estarán mejor explicadas en un docuemento adjunto.

Ahora procedamos con la construcción del modelo final será el "roberta-base".

Consideraciones:

* Los modelos preentrenados necesitan un learning rate alto, sin embargo, hemos puesto un clasificador propio el cual debemos entrenar con un learning rate mas alto, por tal motivo necesitaremos un learning rate más alto, por tal motivo entrenaremos unas pocas epocas con un alto learning rate y las capas del modelo roberta congeladas, luego entrenaremos con un learning rate más bajo.

* ***language models are few shot learners*** en ese paper se señala la propiedad de trade off que tienen el batch size y el tamaño final de nuestras secuencias de textos, para evitar overfitting trabajaremos con secuencuias largas y un batch size más pequeño de lo usual.

* No se utilizarán embedding pre-entrenados para asegurar que los embeddings capten las caractereristicas de nuestro problema

In [ ]:
!pip install transformers

In [ ]:
import re
import string
import numpy as np
import pandas as pd
import torch
import transformers

import torch.nn as nn

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from transformers import RobertaTokenizer, RobertaModel, AdamW, get_linear_schedule_with_warmup

device = torch.device('cuda')

In [ ]:
train_data = pd.read_csv("../input/aichallenge2020/training.csv")
test_data = pd.read_csv("../input/aichallenge2020/test.csv")

In [ ]:
roberta = RobertaModel.from_pretrained('roberta-base')
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

Puesto que tenemos tweets con 4000 caracteres utilizaremos un max_length = 256 

In [ ]:
tokens_train = tokenizer.batch_encode_plus(
    train_data['tweet'].to_list(),
    max_length = 256,
    pad_to_max_length=True,
    truncation=True,
)

tokens_test = tokenizer.batch_encode_plus(
    test_data['tweet'].to_list(),
    max_length = 256,
    pad_to_max_length= True,
    truncation=True
)

In [ ]:
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])

test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])

test_ids = torch.tensor(test_data['Id'].to_list())
train_y = torch.tensor(train_data['is_organico'].to_list())

Ya que utilizamos un longitud maxima utilizamos un batch size pequeño

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#define a batch size
batch_size = 8

# wrap tensors
train_dataset = TensorDataset(train_seq, train_mask, train_y)
# sampler for sampling the data during training
train_sampler = RandomSampler(train_dataset)
# dataLoader for train set
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size, num_workers=4, pin_memory=True)

#Test data
test_dataset = TensorDataset(test_seq, test_mask, test_ids)
#Sampler
test_sampler = SequentialSampler(test_dataset)
#Loader
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=batch_size, num_workers=4)

In [ ]:
class ROBERT_Arch(nn.Module):
  # Model with extra layers on top of RoBERTa
    def __init__(self, dropout_rate=0.3):
        super(ROBERT_Arch, self).__init__()
        
        self.roberta = RobertaModel.from_pretrained('roberta-base')
        self.d1 = torch.nn.Dropout(dropout_rate)
        self.l1 = torch.nn.Linear(768, 64)
        self.bn1 = torch.nn.LayerNorm(64)
        self.d2 = torch.nn.Dropout(dropout_rate)
        self.l2 = torch.nn.Linear(64, 2)
        
    def forward(self, input_ids, attention_mask):
        _, x = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        x = self.d1(x)
        x = self.l1(x)
        x = self.bn1(x)
        x = torch.nn.Tanh()(x)
        x = self.d2(x)
        x = self.l2(x)
        
        return x

In [ ]:
model = ROBERT_Arch(0.4)
model.to(device)

In [ ]:
from transformers import AdamW

# define the optimizer
params = list(model.named_parameters())
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]  
optmizer_parameters = [
                       {"params":[p for n, p in params if not any(nd in n for nd in no_decay)],
                       "weight_decay": 0.001,},
                       {"params":[p for n, p in params if any(nd in n for nd in no_decay)],
                       "weight_decay": 0.0,},
                       
]   

optimizer = AdamW(optmizer_parameters,
                  lr = 1e-4)

Recuerda que hablamos de clases imbalanceadas? pues solucionemos el problema

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

#compute the class weights
class_weights = compute_class_weight('balanced', np.unique(train_data['is_organico'].values), train_data['is_organico'].values)

print("Class Weights:", class_weights)

In [ ]:
from transformers import get_linear_schedule_with_warmup

weights= torch.tensor(class_weights, dtype=torch.float)
epochs = 6
steps = len(train_dataloader)
# push to GPU
weights = weights.to(device)

# define the loss function
cross_entropy = nn.CrossEntropyLoss(weight=weights)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=steps, num_training_steps=steps*12)

In [ ]:
def train(optimizer):
  
  model.train()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save model predictions
  total_preds=[]
  
  # iterate over batches
  for step,batch in enumerate(train_dataloader):
    
    # progress update after every 50 batches.
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    # push the batch to gpu
    batch = [r.to(device) for r in batch]
 
    sent_id, mask, labels = batch

    # clear previously calculated gradients 
    model.zero_grad()        

    # get model predictions for the current batch
    preds = model(sent_id, mask)
   

    # compute the loss between actual and predicted values
    loss = cross_entropy(preds, labels)

    # add on to the total loss
    total_loss = total_loss + loss.item()

    # backward pass to calculate the gradients
    loss.backward()

    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # update parameters
    optimizer.step()
    scheduler.step()

    # model predictions are stored on GPU. So, push it to CPU
    preds=preds.detach().cpu().numpy()

    # append the model predictions
    total_preds.append(preds)

  # compute the training loss of the epoch
  avg_loss = total_loss / len(train_dataloader)
  
  # predictions are in the form of (no. of batches, size of batch, no. of classes).
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  #returns the loss and predictions
  return avg_loss, total_preds

Entrenamos solo el clasificador del modelo

In [ ]:
for param in model.roberta.parameters():
    param.requires_grad = False

In [ ]:
# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, _ = train(optimizer)
    
    train_losses.append(train_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')

Ahora ya que entrenaremos el modelo completo y está pre entrenado pues, hagamoslo con un learning rate mucho más bajo.

In [ ]:
params = list(model.named_parameters())
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]  
optmizer_parameters = [
                       {"params":[p for n, p in params if not any(nd in n for nd in no_decay)],
                       "weight_decay": 0.001,},
                       {"params":[p for n, p in params if any(nd in n for nd in no_decay)],
                       "weight_decay": 0.0,},
                       
]   

optimizer = AdamW(optmizer_parameters,
                  lr = 3e-6)
epochs = 14

In [ ]:
for param in model.roberta.parameters():
  param.requires_grad = True

In [ ]:
# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, _ = train(optimizer)
    
    train_losses.append(train_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')

In [ ]:
data_dicty = {
    "Id": [],
    "Predicted":[]
    }
with torch.no_grad():
  for batch in test_dataloader:
    batch = [r.to(device) for r in batch]

    seq, mask, id = batch
    output = model(seq, mask)
    preds = np.argmax(output.cpu().detach().numpy(), axis = 1)
    for pred, ids in zip(preds, id):
      data_dicty['Predicted'].append(pred)
      data_dicty['Id'].append(int(ids.item()))

In [ ]:
pd.read_csv('./pr.csv')

In [ ]:
data = pd.DataFrame(data_dicty)
data.to_csv("pre.csv", index=False)

In [ ]:
torch.save(model.state_dict(), 'roberta_saved_weights_finañ.pt')

Función para inferencia en dataset

In [ ]:
def inference(csv_file, x, y, batch_size, model):
    """
    x: [str] Nombre de la columna del datapoint en el dataframe
    y; [str] Nombre de la columna label en el dataframe
    csv_file: [str] ruta del archivo csv del dataset
    """
    data = pd.read_csv(csv_file)
    
    tokens = tokens_train = tokenizer.batch_encode_plus(
    data[x].to_list(),
    max_length = 256,
    pad_to_max_length=True,
    truncation=True
    )
    
    data_seq = torch.tensor(tokens['input_ids'])
    data_mask = torch.tensor(tokens['attention_mask'])
    data_y = torch.tensor(data[y].to_list())
    
    test_dataset = TensorDataset(data_seq, data_mask, data_y)
    #Sampler
    test_sampler = SequentialSampler(test_dataset)
    #Loader
    test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=batch_size)
    
    data_dicty = {
    "label": [],
    "Predicted":[]
    }
    with torch.no_grad():
        
      for batch in test_dataloader:
        batch = [r.to(device) for r in batch]
        seq, mask, labels = batch
        output = model(seq, mask)
        preds = np.argmax(output.cpu().detach().numpy(), axis = 1)
        
        for pred, label in zip(preds, labels):
          data_dicty['Predicted'].append(pred)
          data_dicty['label'].append(int(label.item()))
          
      return data_dicty